In [ ]:
!python -m pip install --user -r ../requirements.txt

In [ ]:
!pip install pyarrow

In [ ]:
!curl "https://echanges.dila.gouv.fr/OPENDATA/ACCO/" | grep "<a" | grep "tar.gz" | cut -d">" -f3 | cut -d"<" -f1 | grep "ACCO" | sed -e "s/^/https:\/\/echanges.dila.gouv.fr\/OPENDATA\/ACCO\//g" | xargs -n 1 curl -O 

In [ ]:
!ls  | grep "ACCO"   | xargs -n 1 tar -xzvf 

In [ ]:
!curl "https://echanges.dila.gouv.fr/OPENDATA/ACCO/Freemium_acco_global_20220417-170000.tar.gz" -o "accords.tar.gz"

In [ ]:
!tar -xzvf "accords.tar.gz"

In [ ]:
!mkdir accords 

In [ ]:
!find acco -name '*.docx' -exec mv {} accords \;  

In [ ]:
import pandas as pd
import docx
import os

In [ ]:
DIR_DOCX = "./accords/"
OUTPUT_FILE_PARQUET = "Dares_accords_docx_sommaire.parquet"

In [ ]:
def read_txt_from_docx(filename: str) -> str:
    fullText = []
    try:
        doc = docx.Document(filename)
        for para in doc.paragraphs:
            for run in para.runs:
                if run.font.bold:
                    fullText.append(run.text)

    except:
        print(f"bug docx {filename}")
    return "\n".join(fullText)

In [ ]:
df = pd.DataFrame(columns=["num_dossier", "accorddocxsommaire"],dtype=object)

for i, num_dossier in enumerate(
    {file_name[:12] for file_name in os.listdir(DIR_DOCX) if file_name.endswith(".docx")}
):
    all_docx_files_for_num_dossier = sorted(
        [
            DIR_DOCX+file_name
            for file_name in os.listdir(DIR_DOCX)
            if file_name.endswith(".docx") and file_name.startswith(num_dossier)
        ],
        key=os.path.getsize,
        reverse=True,
    )

    new_line = [num_dossier] + [
        read_txt_from_docx(file) for file in all_docx_files_for_num_dossier
    ]
    new_line = new_line[:2] if len(new_line) >= 2 else new_line + [""]
    try:
        new_line = {
            "num_dossier": new_line[0],
            "accorddocxsommaire": new_line[1],
            "index" : [i]
        }
    except:
        print(f"bug {num_dossier}")
        
    
    df_new_row = pd.DataFrame.from_records(data=new_line)
    df = pd.concat([df, df_new_row])

In [ ]:
df.to_parquet(OUTPUT_FILE_PARQUET)